In [ ]:
gitdifftool--tool = b3branch1..branch2$gitdiff > save.patch$patch - p1 < save.patch$gitdiff--no - prefix > save.patch$patch - p0 < save.patch

Rebase squash cherrypick automatically to create perfect feature branch?

Reset each file one at a time to make sure there aren't any unnecessary changes?

Use unit testing to warn about unnecessary modules?

Load relevant projects based on branch.  It could parse the .sln file and automatically patch whenever the checked out index changes.





In [ ]:
gitlog--graph--oneline--decorate--allgitlog--graph--allgitshow - branch--listhttps://stackoverflow.com/questions/3515597/add-only-non-whitespace-changes    gitdiff - U0 - w--no - color | gitapply--cached--ignore - whitespace--unidiff - zero -

Rename a bunch of files using Git and renamer?



In [ ]:
var execSync = require('child_process').execSync;try {    require.resolve('renamer');} catch (e) {    execSync('npm install renamer');}var renamer = require('renamer');var renameUsingGit = (gitRoot, match, find, replace) => {    var files = renamer.expand(path.join(gitRoot, match));    var results = renamer.replace({        files: files.filesAndDirs,        find: find,        replace: replace    });    return renamer.dryRun(results).list.map(r => {        // rename with git instead        var output = execSync('git '            + '--work-tree=' + JSON.stringify(gitRoot)            + ' --git-dir=' + JSON.stringify(path.join(gitRoot, '.git'))            + ' mv ' + JSON.stringify(r.before)            + ' ' + JSON.stringify(r.after));        return output.toString() || r.after;    });};renameUsingGit;

In [ ]:
// test it// rename all scss files to less//renameUsingGit(//    '/Users/briancullinan/Documents/studysauce4',//    'src/**/*.css',//    '.css',//    '.scss');

Create a json tree out of git?



In [ ]:
var execSync = require('child_process').execSync;var gitTree = (project) => {    process.chdir(project)    var branches = execSync('git branch --list --all ').toString()        .trim().split(/\s*\n\s*\**\s*/ig)        .map(b => b.replace('remotes/', '').trim());    // reverse git so we can add merges to the "graph" after the branch tree is constructed    var branch = execSync('git log --pretty=format:"%ct %d %h %p" '        + '--reverse --branches --tags ').toString().split(/\s*\n\s*/ig);    var all = []; //execSync('git log --pretty=format:"%h %p" --reverse --all').toString().split(/\s*\n\s*/ig);    var nodeTree = [];    var edges = [];    branch.forEach(b => {        var commits = ((/[0-9]+\s+\(.*?\)\s*(.+)/ig).exec(b)            || (/[0-9]+\s+(.+)/ig).exec(b) || [b, b.trim()])[1].trim().split(/\s+/ig);        commits.forEach(r => {            if (nodeTree.filter(d => d.name == r).length == 0) {                nodeTree[nodeTree.length] = {                    name: r,                    index: nodeTree.length,                    branch: ((/[0-9]+\s+\((.*?)\)/ig).exec(b) || [b, ''])[1],                    time: (/([0-9]+)\s+/ig).exec(b)[1]                }            }        });        // don't include merges in tree        if (commits.length == 3) {            commits = [commits[0], commits[2]];        }        if (commits.length == 2) {            edges = edges.concat(commits.slice(1)            // check all existing sources to test for merge                .map((c, i) => {                    return {source: commits[i], target: c}; // -1 because of slice(1)                }))        }    });    var nodeMap = {};    nodeTree.forEach(function (x) {        if (typeof x.size == 'undefined') {            x.size = 1;        }        nodeMap[x.name] = x;    });    edges.forEach(function (x) {        if (typeof nodeMap[x.target].children == 'undefined') {            nodeMap[x.target].children = [];        }        nodeMap[x.source].parent = nodeMap[x.target];        nodeMap[x.target].children.push(nodeMap[x.source]);        // return { parent: nodeMap[x.target], source: nodeMap[x.source], target: nodeMap[x.target] };    });    edges.reverse().forEach(function (x) {        if (typeof nodeMap[x.source].branch != 'undefined' && nodeMap[x.source].branch != ''            && typeof nodeMap[x.target].branch == 'undefined' || nodeMap[x.target].branch == '') {            nodeMap[x.target].branch = nodeMap[x.source].branch;        }    });    return nodeTree;}module.exports = gitTree;gitTree;

How to display git branches as a tree?



In [ ]:
var D3Node = require('d3-node');var margin = {top: 25, right: 300, bottom: 25, left: 25},    width = 960 - margin.left - margin.right,    height = 500 - margin.top - margin.bottom;var displayBranches = (nodeTree) => {    var d3n = new D3Node(); // initializes D3 with container element     var d3 = d3n.d3;    var tree = d3.hierarchy(nodeTree[0], d => d.children);    var gitRoot = // partition(        tree            .sum(d => 1)            .sort((a, b) => b.data.time - a.data.time || b.value - a.value)            //)            .descendants();    var branches = gitRoot        .map(r => r.data.branch.replace('HEAD -> ', '').trim())        .filter((b, i, arr) => arr.indexOf(b) === i)    gitRoot.forEach(r => {        r.name = r.data.name;        r.index = r.data.index;        r.branch = r.data.branch;    })    var tree = d3.tree().nodeSize([25, 150]);    var nodes = tree(d3.hierarchy(gitRoot[0], d => d.children))        .descendants();    var links = nodes.slice(1);    var fill = d3.scaleOrdinal(d3.schemeCategory20);    var svg = d3n.createSVG(        branches.length * 20 + margin.left + margin.right,        (nodes.length + 1) * 20 + margin.top + margin.bottom)    var g = svg.append('g');    var x = d3.scaleLinear().range([branches.length * 20, 0]);    var y = d3.scaleLinear().range([(nodes.length + 1) * 20, 0]);    x.domain([0, branches.length]);    y.domain([0, nodes.length]);    var branchIndex = (d) => {        var result = Math.min.apply(null, d.data.branch            .split(/\s*,\s*/ig)            .map(b => branches.indexOf(b))            .filter(i => i > -1));        if (!isFinite(result)) {            result = 0;        }        return result;    };    g.selectAll('.link')        .data(links)        .enter().append('path')        .attr('class', 'link')        .attr('fill', 'none')        .attr('stroke', '#000')        .attr('stroke-width', 2)        .attr('d', d => `            M${x(branchIndex(d))},${y(d.data.index)}            C${x((branchIndex(d) + branchIndex(d.parent)) / 2.0)},              ${y(d.data.index)} ${x((branchIndex(d) + branchIndex(d.parent)) / 2.0)},              ${y(d.parent.data.index)} ${x(branchIndex(d.parent))},              ${y(d.parent.data.index)}`);    // Declare the nodes    var node = g.selectAll('g.node')        .data(nodes)        .enter().append('g')        .attr('class', 'node')        .style('fill', function (d, i) {            return fill(d.data.branch);        })        .attr('transform', d => `translate(${x(branchIndex(d))},${y(d.data.index)})`);    node.append('circle')        .attr('class', d => (d.data).isAux ? 'node-aux-route' : 'node-route')        .attr('r', 6);    node.append('text')        .attr('x', (d) => 13)        .attr('dy', '.35em')        .attr('text-anchor', 'start')        .text(d => (d.data).name + (d.data.branch && (!d.data.children || d.data.children.filter(            c => c.data.branch == d.data.branch) == 0) ? (' ( ' + d.data.branch + ' ) ') : ''))        .attr('class', 'monospace');    // reset transform    g.attr('transform', 'translate(0, 0)');    g.attr(        'transform',        'translate(' + (margin.left) + ',' + (margin.top) + ')')    return d3n.svgString();}module.exports = displayBranches;displayBranches;

In [27]:
var importer = require('../Core');var _ = require('underscore');var fs = require('fs');var path = require('path');var execSync = require('child_process').execSync;var gulp = require('gulp');var copy = require('gulp-copy');var clean = require('gulp-clean');var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;var project_orig = PROFILE_PATH + '/Documents/portal';var project = PROFILE_PATH + '/Documents/portal3';process.chdir(PROFILE_PATH + '/Documents');gulp.task('clear project', function () {    return gulp.src(project, {read: false})        .pipe(clean({force: true}));});gulp.task('copy index', function () {    return gulp.src(project_orig)        .pipe(copy(project))        .dest(project);});var getResetCommit = (branch, parent) => {    const b = branch.replace('HEAD -> ', '').trim();    execSync('git checkout ' + b);    execSync('git reset --mixed ' + parent.replace('HEAD -> ', '').trim());    execSync('git add -A')    execSync('git commit -m "auto reset ' + b + '"')}var resetAllBranches = () => {    var children = [];    var nodeTree = _.sortBy(gitTree(project), a => a.time);    var promises = [];    for (const n of nodeTree) {        var current = n;        var parentCount = 0;        while (typeof current.parent !== 'undefined'        && current.parent.branch === current.branch) {            current = current.parent;            parentCount++;        }        if (parentCount > 0 && typeof current.parent !== 'undefined') {            promises.push(resolve => {                try {                    if (typeof n.children !== 'undefined') {                        children.push(n);                    }                    getResetCommit(n.branch, current.parent.name);                } finally {                    var nodeTree = _.sortBy(gitTree(project), a => a.time);                    svgs.push(displayBranches(nodeTree));                    resolve();                }            });        }    }    return importer.runAllPromises(promises)        .then(() => children);}var gitTree, displayBranches, tasksToPromise, listInProject;var svgs = [];var gitTipOfTree = () => {    svgs = [];    return importer.import([        'gulp tasks',        'json git tree',        'display git branches tree',        'list files project'    ], {gulp})        .then(r => {            tasksToPromise = r[0];            gitTree = r[1];            displayBranches = r[2];            listInProject = r[3];            if (fs.existsSync(project)) {                return tasksToPromise(['clear project']);            }        })        .then(() => Promise.all([            listInProject(project_orig, '**/*'),            listInProject(project_orig + '/.git', '**/*')        ]))        .then(r => {            for (file of [].concat(...r).sort()) {                const dirname = path.dirname(                    file.replace(project_orig, project)).split('/');                for (var i = 2; i <= dirname.length; i++) {                    if (!fs.existsSync(dirname.slice(0, i).join('/'))) {                        fs.mkdirSync(dirname.slice(0, i).join('/'));                    }                }                fs.copyFileSync(file, file.replace(project_orig, project));            }            process.chdir(project);        })        .then(() => {            var nodeTree = _.sortBy(gitTree(project), a => a.time);            return displayBranches(nodeTree);        })        .then(r => svgs.push(r))        .then(() => resetAllBranches())        .then(() => svgs)};module.exports = gitTipOfTree;

[Function: gitTipOfTree]

In [45]:
$$.async();gitTipOfTree()    .then(r => $$.html(`<style>div.svg-wrapper svg {max-width:none;}div.svg-output {min-height:600px;}div.svg-wrapper {opacity: .05;position:absolute;}div.svg-wrapper:nth-last-of-type(3) {opacity: .15;}div.svg-wrapper:nth-last-of-type(2) {opacity: .3;}div.svg-wrapper:last-of-type {opacity: 1;}</style><div class="svg-output"><div` + r.map((s, i) => ' style="left:' + ((r.length - i) * -20 + (i == r.length - 1        ? 200        : 0)        + (i == r.length - 2            ? 150            : 0)        + (i == r.length - 3            ? 100            : 0)) + 'px; top:' + (i * 10) + 'px;" class="svg-wrapper">' + s).join('</div><div') + '</div></div>'))    .catch(e => $$.sendError(e))

Switched to branch 'Universal'
Already on 'Universal'
Already on 'Universal'
Already on 'Universal'
Already on 'Universal'
Already on 'Universal'
Already on 'Universal'
Already on 'Universal'
Already on 'Universal'
Already on 'Universal'
Switched to branch 'B-11376_Account_Management_2'
Already on 'B-11376_Account_Management_2'
Already on 'B-11376_Account_Management_2'
Note: checking out 'swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 69ea6ff... Update preview model to match the new simpler model api is returning now
any of your branches:

  9f72a55 auto res

4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) fee839d a229417 ( swiftpage/development ) 6a552aa a8c1fd7 8fe6b3d 2c184e7 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model ) 081e6d7 67b6565 7fa4cf6 2e2b2f1 6141fc7 fe71b6f ( swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue ) a57a764 ( B-11376_Account_Management_2 ) 580ee75 bcc334f 8c093e1 ( Only_auth ) 8038f45 da4807a 86fcb7e 254ffb0 ( HEAD -> B-11309_APC_Direct_Purchase ) 4eac003 498d5a5 620fcf1 ( Universal ) 4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) a229417 ( swiftpage/development ) a8c1fd7 8fe6b3d 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model ) 081e6d7 7fa4cf6 2e2b2f1 fe71b6f ( swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue ) a57a764 ( B-11376_Account_Management_2 ) 13a3d5f ( HEAD -> Universal ) bcc334f 8c093e1 ( Only_auth ) da4807a 254ffb0 ( B-11309_APC_Direct_Purchase ) 4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) a229417 ( swiftpage/development ) a8c1fd7 8fe6b3d 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model ) 081e6d7 7fa4cf6 2e2b2f1 fe71b6f ( swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue ) a57a764 ( B-11376_Account_Management_2 ) 13a3d5f ( HEAD -> Universal ) bcc334f 8c093e1 ( Only_auth ) da4807a 254ffb0 ( B-11309_APC_Direct_Purchase ) 4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) a229417 ( swiftpage/development ) a8c1fd7 8fe6b3d 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model ) 081e6d7 7fa4cf6 2e2b2f1 fe71b6f ( swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue ) a57a764 ( B-11376_Account_Management_2 ) 13a3d5f ( HEAD -> Universal ) bcc334f 8c093e1 ( Only_auth ) da4807a 254ffb0 ( B-11309_APC_Direct_Purchase ) 4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) a229417 ( swiftpage/development ) a8c1fd7 8fe6b3d 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model ) 081e6d7 7fa4cf6 2e2b2f1 fe71b6f ( swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue ) a57a764 ( B-11376_Account_Management_2 ) 376abec ( HEAD -> Universal ) bcc334f 8c093e1 ( Only_auth ) da4807a 254ffb0 ( B-11309_APC_Direct_Purchase ) 4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) a229417 ( swiftpage/development ) a8c1fd7 8fe6b3d 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model ) 081e6d7 7fa4cf6 2e2b2f1 fe71b6f ( swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue ) a57a764 ( B-11376_Account_Management_2 ) 376abec ( HEAD -> Universal ) bcc334f 8c093e1 ( Only_auth ) da4807a 254ffb0 ( B-11309_APC_Direct_Purchase ) 4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) a229417 ( swiftpage/development ) a8c1fd7 8fe6b3d 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model ) 081e6d7 7fa4cf6 2e2b2f1 fe71b6f ( swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue ) a57a764 ( B-11376_Account_Management_2 ) 376abec ( HEAD -> Universal ) bcc334f 8c093e1 ( Only_auth ) da4807a 254ffb0 ( B-11309_APC_Direct_Purchase ) 4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) a229417 ( swiftpage/development ) a8c1fd7 8fe6b3d 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update

Display graphs of different commands and scenarios using dry-run?

https://github.com/kpdecker/jsdiff

http://incaseofstairs.com/jsdiff/



In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;var importer = require('../Core');process.chdir(project)var gulp = require('gulp');var run = require('gulp-run');var tap = require('gulp-tap');gulp.task('git watch', function () {    return run('git diff -w a229417..498d5a5')        .pipe(tap(file => console.log(file)))        .pipe(gulp.dest('output'));});var getBranchBoundaries = (p) => {    project = p;    return importer.import('gulp task to a Promise', {gulp})        .then(gulpPromise => gulpPromise(['git watch']))}module.exports = getBranchBoundaries;getBranchBoundaries;$$.async();getBranchBoundaries(project)    .then(r => $$.sendResult(r))    .catch(e => $$.sendError(e));